Aqui irei utilizar as funções que desenvolvi no outro notebook e desenvolver o projeto

In [ ]:
# Importando bibliotecas

import time
import pandas as pd

from urllib.error import URLError, HTTPError
from urllib.request import Request, urlopen

import bs4

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# Informando o AGENTE

agente = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
headers = {'User-Agent': agente}

In [ ]:
# Chamar a página WEB

def ConsultaWebB(url):
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        return response.read()

    except:
        pass

In [ ]:
# Pegar a página encontrada e transformar em um objeto utilizável

def captura_html_pagina(url):
    html = ConsultaWebB(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')    
    return soup

In [ ]:
# Capturar as informações da "tabelinha cabeçalho" da página

def Cabecalho(html):
    dt = html.find_all('dt')
    dd = html.find_all('dd')
    dic = {}
    for i in range(len(dt)):
        x = dt[i].get_text()
        y = dd[i].get_text() 
        dic[x] = y
    return dic

In [ ]:
# Capturar o conteúdo da página e transformar em um dicionário

def Conteudo(proposicao, ano):
    url = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao='+str(proposicao)+'&aaProposicao='+str(ano)
    html = captura_html_pagina(url)
    dic = Cabecalho(html)
    dic['Proposição'] = proposicao
    dic['Ano'] = ano
    dic['Texto'] = html.p.get_text()
    return dic

In [ ]:
def TabelaResultados(inicar_em, quantidade, ano, erros_admissiveis, segundos_espera):

    ultima_consulta = inicar_em + quantidade - 1

    # erros
    erros = 0

    # variaveis para loop
    i = 1
    lista = []

    while inicar_em <= ultima_consulta and erros <= erros_admissiveis:

        try:
            x = Conteudo(inicar_em,ano)
            lista = lista + [Conteudo(inicar_em,ano)]
        except:
            erros += 1
            pass

        time.sleep(segundos_espera)

        # carregamento incremental das variáveis
        inicar_em+=1 
        i+=1


    return pd.DataFrame(lista)

In [ ]:
teste = TabelaResultados(500,3,2021,2,0.2)
teste

# <font color=green> 2. CRIANDO O BANCO DE DADOS E TABELAS

> Primeiramente você precisa instalar o ``SQL Server`` e o ``Management Studio``

In [ ]:
# Lista com nome de colunas
list(teste.columns)

In [ ]:
# Lista com tipos de dados
teste.dtypes

>  Montei o script, seguindo esse padrão:

```sql
CREATE TABLE Proposicoes (

	DataReuniao DATE,
	DataDeliberacao DATE,
	Situacao VARCHAR(200),
	Assunto VARCHAR(1000),
	Autor VARCHAR(1000),
	Proposicao INT,
	Ano INT,
	Texto VARCHAR(MAX)

)
```

### 2.1 Alimentando Tabelas

In [ ]:
# Biblioteca que conecta o Python ao SQL
import pyodbc

In [ ]:
pyodbc.version
# pip install pyodbc==4.0.30

Conectando ao banco

In [ ]:
'''
# Com senha
conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'localhost', 
                      database = 'Indaial',
                      UID='sa',
                      PWD='SuaSenha')
'''

# Com autenticação do Windows
conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'localhost', 
                      database = 'Indaial')

query = '''
    select 
        * 
    from Proposicoes
'''
sql_query = pd.read_sql_query(query,conn)
sql_query

In [ ]:
# Crio uma base
base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'])
base

In [ ]:
# "Combino" as bases
base = base.append(teste)
base

In [ ]:
base.fillna('')

In [ ]:
base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'])
teste = base.append(teste).fillna('')

conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'localhost', 
                      database = 'Indaial')

cursor = conn.cursor()

for index, row in teste.iterrows():
    
    cursor.execute('''
    
        INSERT INTO Proposicoes (
            DataReuniao,
            DataDeliberacao,
            Situacao,
            Assunto,
            Autor,
            Proposicao,
            Ano,
            Texto
        ) 
        values(?,?,?,?,?,?,?,?)''', # mesma quantidade de variáveis

        row['Reunião'], 
        row['Deliberação'], 
        row['Situação'], 
        row['Assunto'], 
        row['Autor'], 
        row['Proposição'], 
        row['Ano'], 
        row['Texto']

    )

conn.commit()
cursor.close()

> E então...

In [ ]:
conn = pyodbc.connect('Trusted_Connection=yes', 
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'localhost', 
                      database = 'Indaial')

query = '''
    select 
        * 
    from Proposicoes
'''
sql_query = pd.read_sql_query(query,conn)
sql_query

In [ ]:
def SQLSelect(query):
    
    conn = pyodbc.connect('Trusted_Connection=yes', 
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost', 
                          database = 'Indaial')

    out = pd.read_sql_query(query,conn)
    return out

In [ ]:
SQLSelect('''
    select 
        *
    from Proposicoes
''')

> Crio uma função para fazer a inserção dos dados na tabela ``Proposicoes``.

In [ ]:
def SQLInsertProposicoes(TabelaProposicoes):
    
    base = pd.DataFrame(columns=['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição', 'Ano', 'Texto'])
    TabelaProposicoes = base.append(TabelaProposicoes).fillna('')

    conn = pyodbc.connect('Trusted_Connection=yes', 
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost', 
                          database = 'Indaial')

    cursor = conn.cursor()

    for index, row in TabelaProposicoes.iterrows():

        cursor.execute('''

            INSERT INTO Proposicoes (
                DataReuniao,
                DataDeliberacao,
                Situacao,
                Assunto,
                Autor,
                Proposicao,
                Ano,
                Texto
            ) 
            values(?,?,?,?,?,?,?,?)''', 

            row['Reunião'], 
            row['Deliberação'], 
            row['Situação'], 
            row['Assunto'], 
            row['Autor'], 
            row['Proposição'], 
            row['Ano'], 
            row['Texto']

        )

    conn.commit()
    cursor.close()

In [ ]:
SQLInsertProposicoes(teste)

In [ ]:
SQLSelect('''
    select 
        *
    from Proposicoes
''')

In [ ]:
def SQLTruncate(NomeTabela):

    conn = pyodbc.connect('Trusted_Connection=yes', 
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost', 
                          database = 'Indaial')

    cursor = conn.cursor()

    cursor.execute(f'''

                   TRUNCATE TABLE {NomeTabela}

                   ''')

    conn.commit()
    cursor.close()

In [ ]:
# Limpando dados da tabela
SQLTruncate('Proposicoes')

# Visualizando a tabela
SQLSelect('select *from Proposicoes')

> Agora que sei CRIAR, LIMPAR, e SELECIONAR dados de uma tabela via SQL, posso automatizar a inserção de dados nela

# <font color=green> 3. ALIMENTAÇÃO INCREMENTAL

### 3.1 MODIFICANDO A FUNÇÃO DE BUSCA

In [ ]:
# Limpar os dados da tabela só pra demonstração
#SQLTruncate('Proposicoes')


# ========= INICIO =========
proposicao = 203
ano = 2021
dados = Conteudo(proposicao,ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)
# ========= FIM =========


# visualizando dados
SQLSelect(f'select * from Proposicoes where Proposicao = {proposicao} and Ano = {ano}')

> Pensando em atualização incremental, cada consulta, tem que ser ``proposicao_anterior + 1``

In [ ]:
# ========= INICIO =========
ano = 2021

dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
ultima_proposicao = dados_ano['Proposicao'].loc[0]

proxima_proposicao = int(ultima_proposicao) + 1

dados = Conteudo(proxima_proposicao,ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)
# ========= FIM =========


# visualizando dados
SQLSelect('select * from Proposicoes')

In [ ]:
# Limpando dados da tabela
SQLTruncate('Proposicoes')

# Visualizando a tabela
SQLSelect('select *from Proposicoes')

Porém a fórmula acima apresenta um problema, caso a tabela esteja vazia.

Então para corrigir isso

In [ ]:
# ========= INICIO =========
ano = 2021

dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
ultima_proposicao = dados_ano['Proposicao'].loc[0]


# ----- INICIO ALTERAÇÃO ----- 
if ultima_proposicao == None:
    proxima_proposicao = 1
else:
    proxima_proposicao = int(ultima_proposicao) + 1 
# ----- FIM ALTERAÇÃO ----- 


dados = Conteudo(proxima_proposicao,ano)
tabela = pd.DataFrame([dados])
SQLInsertProposicoes(tabela)
# ========= FIM =========


# visualizando dados
SQLSelect('select * from Proposicoes')

Essa função vai funcionar muito bem enquanto não chega na ultima proposição.


In [ ]:
def InsereProximaProposicao(ano):

    # Busca última proposição cadastrada
    dados_ano = SQLSelect(f'select Proposicao = max(Proposicao) from Proposicoes where Ano = {ano}')
    ultima_proposicao = dados_ano['Proposicao'].loc[0]

    # Verifica se foi identificado lançamento naquele ano
    if ultima_proposicao == None:
        proxima_proposicao = 1
    else:
        proxima_proposicao = int(ultima_proposicao) + 1 

    # Captura e Insere dados na tabela
    dados = Conteudo(proxima_proposicao,ano)
    tabela = pd.DataFrame([dados])
    SQLInsertProposicoes(tabela)

In [ ]:
InsereProximaProposicao(1996)
SQLSelect('select *from Proposicoes')

In [ ]:
# Limpando dados da tabela
SQLTruncate('Proposicoes')

# Visualizando a tabela
SQLSelect('select *from Proposicoes')

In [ ]:
def BuscaGravaDadosAno(ano, quantidade = 10, erros_admissiveis = 2, segundos_espera = 0):

    # erros
    erros = 0

    # variaveis para loop
    i = 1
    lista = []

    while erros <= erros_admissiveis:

        try:
            InsereProximaProposicao(ano)
        except:
            erros += 1
            pass

        time.sleep(segundos_espera)

        # carregamento incremental das variáveis
        i +=1

In [ ]:
# Limpando dados da tabela
SQLTruncate('Proposicoes')

# Visualizando a tabela
SQLSelect('select *from Proposicoes')

In [ ]:
ano_inicial = 1996
ano_final = 2021

for i in list(range(ano_inicial, ano_final+1)):
    print('Iniciando gravação dos dados do ano: ',i)
    try:
        BuscaGravaDadosAno(i, quantidade = 999999)
    except:
        pass
    
print('Inserção finalizada')